In [2]:
import pandas as pd
import difflib

In [3]:
vehicle = pd.read_excel(r'C:\Users\Brice\travel-studies\2017\data-challenge\2017-internal1-R-3-vehicle.xlsx',
                    sheetname='3-Vehicle')

In [4]:
emissions = pd.read_csv(r'C:\Users\Brice\travel-studies\2017\data-challenge\epa-mileage.csv')

# Assume all vehicles are automatic
# Exclude any rows with Manual in title

emissions = emissions[emissions['trany'].apply(lambda row: False if 'Manual' in str(row) else True) == True]

# May still need to take average for these transmission types if there are multiple types of automatics

In [335]:
# Perform some cleanup on the vehicle file from survey to increase matches with EPA data
# Earliest year from EPA data is 1984. For older vehicles, try to find the oldest make/model/year combination
df = vehicle.copy()
for index, row in vehicle[vehicle['year'] < 1984].iterrows():
    make = row['make']
    model = row['model']
    min_year = emissions[(emissions['model'] == model) & (emissions['make'] == make)]['year'].min()
    
    # replace year with min_year from emissions data for a match
    df.ix[index,'year'] = min_year

In [5]:
emissions[emissions['year'] < 1984][['make','model','year']]

,make,model,year


In [340]:
make = 'Chevrolet'
model = 'Camaro'
emissions[(emissions['model'] == model) & (emissions['make'] == make)]['year'].min()

1984

In [286]:
# Get a unique list of vehicles from survey
make_model_df = vehicle.groupby(['make','model','year']).count().reset_index()[['make','model','year']]

df_result = pd.DataFrame()

# loop through these vehicles and add any rows that match 
i = 0
for i in xrange(len(make_model_df)):
    model = make_model_df.iloc[i]['model']
    make = make_model_df.iloc[i]['make']
    year= make_model_df.iloc[i]['year']
    
    # Select year and make based on the vehicles
    _df = emissions[(emissions['year'] == year) & (emissions['make'] == make)]
    
    # these should contain some of the models
    # loop through each, test if the value of model from vehicles file is contained in the emissions file
    # add each partial match from emissions file
    index_list = []
    for i in xrange(len(_df)):    # loop through each match
        # If multiple fields in model name in vehicle file, split and search for each
        for part in model.split(' '):
            if part in _df.iloc[i]['model']:
                index_list.append(i)
    
    # Export matching rows
    _df = _df.iloc[index_list]
    # Add a field from vehicle model to merge later
    _df['model_common'] = model
    
    df_result = df_result.append(_df)
    i+=1
    

In [288]:
# df_result

In [289]:
_df.iloc[0]['model'].split(' ')

['fortwo', 'coupe']

In [290]:
# We are missing some vehicles that don't have an 

In [291]:
# Result will still have multiple results per make/model/year because of different engine types or model differences
# group by model_common and take averages
average_cols = ['highway08','co2TailpipeGpm','comb08','city08']    
first_cols = []
df = df_result.groupby(['year','make','model_common']).mean()[average_cols].reset_index()

_df = df_result.groupby(['year','make','model_common']).first()[['fuelType','VClass','model']].reset_index()
_df.rename(columns={'model':'model_epa'},inplace=True)

df = pd.merge(df,_df)

# Join the trimmed down results to the vehicle data
df = pd.merge(vehicle, df, left_on=['year','make','model'], right_on=['year','make','model_common'], how='left')

In [298]:
# Get list of unpaired models
_df = df[df['model_epa'].isnull()]
_df.groupby(['make','model']).count()

hhid  vehnum  vehid  year  fuel  disability  \
make          model                                                          
Acura         CSX                 1       1      1     1     1           1   
              Other               1       1      1     1     1           1   
Alfa Romeo    Spider              1       1      1     1     1           1   
Audi          Other               1       1      1     1     1           1   
BMW           Other               9       9      9     9     9           9   
Cadillac      Deville             1       1      1     1     1           1   
Chevrolet     C3500               2       2      2     2     2           2   
              Camaro              1       1      1     1     1           1   
              K2500               1       1      1     1     1           1   
              Other               8       8      8     8     8           8   
              Silverado           1       1      1     1     1           1   
Chrysler      Neon                1       1      1     1     1           1   
Dodge         Other               4       4      4     4     4           4   
              Sprinter 2500       2       2      2     2     2           2   
              Stratus             1       1      1     1     1           1   
Fiat          500L                2       2      2     2     2           2   
              500c                1       1      1     1     1           1   
Ford          E-350               2       2      2     2     2           2   
              E-450               2       2      2     2     2           2   
              E450 Super Duty     2       2      2     2     2           2   
              Econoline           1       1      1     1     1           1   
              Excursion           1       1      1     1     1           1   
              F-150              44      44     44    44    44          44   
              F-250              16      16     16    16    16          16   
              F-350               9       9      9     9     9           9   
              Other              10      10     10    10    10          10   
GMC           Other               2       2      2     2     2           2   
Honda         Other              14      14     14    14    14          14   
              S2000               1       1      1     1     1           1   
Hyundai       Other               1       1      1     1     1           1   
Jaguar        XK8                 1       1      1     1     1           1   
Jeep          Other               1       1      1     1     1           1   
Kia           Forte5              1       1      1     1     1           1   
              Rio5                2       2      2     2     2           2   
              Sedona              1       1      1     1     1           1   
              Spectra5            2       2      2     2     2           2   
Lexus         NX200t              2       2      2     2     2           2   
              Other               3       3      3     3     3           3   
MINI          Other               2       2      2     2     2           2   
Mazda         Mazda2              6       6      6     6     6           6   
              Mazda3             83      83     83    83    83          83   
              Mazda5             11      11     11    11    11          11   
              Mazda6             16      16     16    16    16          16   
              Other              11      11     11    11    11          11   
              Protege5            1       1      1     1     1           1   
Mercedes-Benz 300                 1       1      1     1     1           1   
              C-Class             7       7      7     7     7           7   
              CLA-Class           2       2      2     2     2           2   
              CLK-Class           3       3      3     3     3           3   
              E-Class             5       5      5   

In [299]:
_df[_df['model'] == 'Camaro']

,hhid,vehnum,vehid,year,make,model,fuel,disability,purchase_date,model_common,highway08,co2TailpipeGpm,comb08,city08,fuelType,VClass,model_epa
4183,17153243,3,1715324303,1982,Chevrolet,Camaro,1,1,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
emissions['year'].min()

1984

In [ ]:
# Before joining, make a list of revisions that should be changed slightly to match
# For instance - Mazda3 from vehicles file should be changed to 3
# Similarly, for Ford vehicles change F-150 to F150

In [295]:
df[['make','model','model_common','model_epa']]

,make,model,model_common,model_epa
0,Subaru,Forester,Forester,Forester AWD
1,Honda,Civic,Civic,Civic
2,Honda,HR-V,HR-V,HR-V 2WD
3,Nissan,Sentra,Sentra,Sentra/200SX
4,Acura,TSX,TSX,TSX
5,Honda,Civic,Civic,Civic Natural Gas
6,Subaru,Other,NaN,NaN
7,BMW,3 series,3 series,325i Convertible
8,Toyota,Previa,Previa,Previa
9,Ford,Mustang,Mustang,Mustang


In [297]:
emissions[emissions['make'] == 'Chevrolet'][['make','model']]

,make,model
35,Chevrolet,Lumina
37,Chevrolet,Lumina
38,Chevrolet,Lumina
39,Chevrolet,Lumina
124,Chevrolet,Caprice
126,Chevrolet,Caprice
127,Chevrolet,Caprice
159,Chevrolet,Cavalier Wagon
160,Chevrolet,Cavalier Wagon
240,Chevrolet,Caprice Wagon


In [247]:
df[['model','model_common','model_epa']]

,model,model_common,model_epa
0,Forester,Forester,Forester AWD
1,Civic,Civic,Civic
2,HR-V,HR-V,HR-V 2WD
3,Sentra,Sentra,Sentra/200SX
4,TSX,TSX,TSX
5,Civic,Civic,Civic Natural Gas
6,Other,NaN,NaN
7,3 series,NaN,NaN
8,Previa,Previa,Previa
9,Mustang,Mustang,Mustang


In [205]:
make = 'Subaru'
model = 'Forester'


In [209]:
_df.iloc[index_list][['make','model']]

,make,model
24148,Subaru,Forester AWD
24150,Subaru,Forester AWD


In [203]:
_df = emissions[(emissions['year'] == year) & (emissions['make'] == make)]

In [207]:
_df[['make','model']]

,make,model
23852,Subaru,Impreza AWD
23897,Subaru,Legacy AWD
23899,Subaru,Legacy AWD
23971,Subaru,Impreza Wagon/Outback Sport AWD
24146,Subaru,Outback Wagon AWD
24148,Subaru,Forester AWD
24150,Subaru,Forester AWD
24151,Subaru,Outback Wagon AWD
24152,Subaru,Tribeca AWD


In [166]:
_df[_df['model'].apply(lambda row: model in row) == True]

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
32735,0.186,0.0,0,3.0,124,124.0,0,0.0,0.0,27.0,...,MBX,NaN,0.0,NaN,Tue Dec 05 00:00:00 EST 2017,Mon Feb 12 00:00:00 EST 2018,N,0,0,0


In [163]:
make_model_df

,make,model,year
0,AM General,DJ Po Vehicle 2WD,1984
1,AM General,FJ8c Post Office,1984
2,AM General,Post Office DJ5 2WD,1985
3,AM General,Post Office DJ8 2WD,1985
4,ASC Incorporated,GNX,1987
5,Acura,2.2CL/3.0CL,1997
6,Acura,2.3CL/3.0CL,1998
7,Acura,2.3CL/3.0CL,1999
8,Acura,2.5TL,1995
9,Acura,2.5TL/3.2TL,1996


In [153]:
# Sometimes there are slight differences in the remaining results
# Differences are based on engine types
# Take the average across similar makes and models
# Maybe there's a way to do a weighted average with purchase data...

df_result.groupby(['year','make','model']).mean()[average_cols]

highway08  \
year make                               model                                           
1984 AM General                         DJ Po Vehicle 2WD                   17.000000   
                                        FJ8c Post Office                    13.000000   
     American Motors Corporation        Eagle 4WD                           19.500000   
                                        Eagle SX/4 4WD                      20.500000   
     Aston Martin                       Lagonda                             10.000000   
                                        Saloon/Vantage/Volante              11.000000   
                                        Saloon/vantage/volante              11.000000   
     Audi                               4000                                24.000000   
                                        5000S                               20.000000   
                                        5000S Wagon                         20.500000   
                                        Coupe GT                            20.500000   
     Avanti Motor Corporation           Avanti II                           21.000000   
     BMW                                3 Series                            24.500000   
                                        5 Series                            23.000000   
                                        6 Series                            22.000000   
                                        7 Series                            22.000000   
     Bill Dovell Motor Car Company      Dovell 230CE                        19.000000   
                                        Dovell 230E                         19.000000   
     Bitter Gmbh and Co. Kg             SC                                  16.000000   
     Buick                              Century                             26.555556   
                                        Century Estate Wagon                25.555556   
                                        Electra Estate Wagon                22.000000   
                                        Electra/Park Avenue                 22.666667   
                                        LeSabre                             20.444444   
                                        Regal                               23.333333   
                                        Riviera                             20.833333   
                                        Riviera Convertible                 19.800000   
                                        Skyhawk                             26.666667   
                                        Skyhawk Wagon                       27.500000   
                                        Skylark                             25.750000   
     Cadillac                           Brougham/DeVille (RWD)              21.333333   
                                        Cimarron                            26.000000   
                                        Eldorado                            22.000000   
                                        Eldorado Convertible                20.000000   
                                        Fleetwood/DeVille (FWD)             31.000000   
                                        Limousine                           10.000000   
                                        Seville                             22.000000   
     Chevrolet                          C10 Pickup 2WD                      18.500000   
                                        C20 Pickup 2WD                      18.600000   
                                        Camaro                              24.000000   
                                        Caprice Wagon                       21.000000   
                                        Cavalier                            27.500000   
                                        Cavalier Convertible                26.000000   
                                        Cavalier Wagon                 

Index([u'barrels08', u'barrelsA08', u'charge120', u'charge240', u'city08',
       u'city08U', u'cityA08', u'cityA08U', u'cityCD', u'cityE', u'cityUF',
       u'co2', u'co2A', u'co2TailpipeAGpm', u'co2TailpipeGpm', u'comb08',
       u'comb08U', u'combA08', u'combA08U', u'combE', u'combinedCD',
       u'combinedUF', u'cylinders', u'displ', u'drive', u'engId', u'eng_dscr',
       u'feScore', u'fuelCost08', u'fuelCostA08', u'fuelType', u'fuelType1',
       u'ghgScore', u'ghgScoreA', u'highway08', u'highway08U', u'highwayA08',
       u'highwayA08U', u'highwayCD', u'highwayE', u'highwayUF', u'hlv', u'hpv',
       u'id', u'lv2', u'lv4', u'make', u'model', u'mpgData', u'phevBlended',
       u'pv2', u'pv4', u'range', u'rangeCity', u'rangeCityA', u'rangeHwy',
       u'rangeHwyA', u'trany', u'UCity', u'UCityA', u'UHighway', u'UHighwayA',
       u'VClass', u'year', u'youSaveSpend', u'guzzler', u'trans_dscr',
       u'tCharger', u'sCharger', u'atvType', u'fuelType2', u'rangeA',
       u'evMotor', u

In [149]:
df[df['model'] == 'Civic'].T

,1,2,3,7,59,81,82,83,84,100,...,5538,5539,5540,5574,5584,5593,5618,5620,5625,5626
hhid,17100005,17100005,17100005,17100108,17100563,17100789,17100789,17100789,17100794,17100907,...,17152671,17152671,17152671,17153116,17153243,17153336,17153506,17153587,17153632,17153632
vehnum,2,2,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2
vehid,1710000502,1710000502,1710000502,1710010801,1710056301,1710078901,1710078901,1710078901,1710079401,1710090701,...,1715267101,1715267101,1715267101,1715311601,1715324301,1715333601,1715350601,1715358701,1715363202,1715363202
year,2003,2003,2003,2013,2010,2004,2004,2004,2011,1998,...,2003,2003,2003,2007,2012,2016,2016,2007,2015,2015
make,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda,...,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda,Honda
model,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic,...,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic,Civic
fuel,3,3,3,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
disability,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,2,1,1,1,1
purchase_date,2003,2003,2003,2013,2010,2015,2015,2015,2011,1998,...,2010,2010,2010,2010,2013,2016,2016,2010,2015,2015
barrels08,10.3003,11.3659,11.3659,10.3003,11.3659,10.3003,11.3659,10.987,11.3659,11.7718,...,10.3003,11.3659,11.3659,11.3659,10.6326,NaN,NaN,11.3659,9.98818,10.3003


In [146]:
pd.set_option('display.max_rows', 1000)
# df_result[['make','model','year']]
print df_result.iloc[0:2].T

                                        19314                         20287
barrels08                             19.3888                       19.3888
barrelsA08                                  0                             0
charge120                                   0                             0
charge240                                   0                             0
city08                                     18                            18
city08U                                     0                             0
cityA08                                     0                             0
cityA08U                                    0                             0
cityCD                                      0                             0
cityE                                       0                             0
cityUF                                      0                             0
co2                                        -1                            -1
co2A        